<a href="https://colab.research.google.com/github/patnaka/data-portfolios/blob/main/chinook-analysis/Chinook_SQL_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('chinook.db')

queries = {
    'total_revenue_customers': '''SELECT
                                    sum(total) as Revenue,
	                                sum(customerid) as Total_Customers
	                            from invoices;''',
    'top_customers_by_spending': '''SELECT
                                    cu.firstname || ' ' || cu.lastname as Customer_name,
                                    round(sum(invi.unitprice * invi.quantity), 2) as Total_spent
                                from customers as cu
                                    JOIN invoices 	    as inv 	on cu.CustomerId = inv.CustomerId
                                    Join invoice_items 	as invi on inv.InvoiceId = invi.InvoiceId
                                GROUP BY 1
                                ORDER BY 2 DESC
                                Limit 10;''',
    'top_selling_artists': '''SELECT
                                ar.name as Artist,
                                sum(invi.quantity) as Units_Sold
                            from artists as ar
                                JOIN albums as al on ar.ArtistId = al.ArtistId
                                Join tracks as tr on al.AlbumId = tr.AlbumId
                                JOIN invoice_items as invi on tr.TrackId = invi.TrackId
                            GROUP BY 1
                            ORDER BY 2 DESC
                            Limit 10;''',
    'top_selling_genres': '''SELECT
                                ge.name as Genre_name,
                                sum(invi.quantity) as Units_Sold
                            from genres as ge
                                Join tracks        as tr   on ge.GenreId = tr.GenreId
                                JOIN invoice_items as invi on tr.TrackId = invi.TrackId
                            GROUP BY 1
                            ORDER BY 2 DESC
                            Limit 10;''',
    'total_songs_of_each_genres': '''SELECT
                                        ge.Name AS Genre_name,
                                        COUNT(tr.TrackId) AS Total_Songs
                                    FROM genres AS ge
                                    JOIN tracks AS tr ON ge.GenreId = tr.GenreId
                                    GROUP BY 1
                                    ORDER by 2 DESC;''',
    'monthly_revenue_trends_2011_2013': '''SELECT
                                                strftime('%Y-%m', InvoiceDate) AS Monthid,
                                                sum(total) as Revenue
                                            from invoices
                                            WHERE strftime('%Y', InvoiceDate) in ('2011', '2012', '2013')
                                            GROUP by 1;''',
    'customer_geography_spend_over_100': '''SELECT
                                                cu.Country AS Customer_country,
                                                round(SUM(invi.UnitPrice * invi.Quantity), 2) AS Total_spent
                                            FROM customers AS cu
                                            JOIN invoices      AS inv  ON cu.CustomerId = inv.CustomerId
                                            JOIN invoice_items AS invi ON inv.InvoiceId = invi.InvoiceId
                                            GROUP BY cu.Country
                                            HAVING Total_spent >= 100
                                            ORDER BY Total_spent DESC;'''
}

for chinook_analyse, query in queries.items():
    df = pd.read_sql_query(query, conn)
    df.to_csv(f'{chinook_analyse}.csv', index=False)